In [4]:
!pip install emoji
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
import string
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.models import Model
from sklearn.metrics import precision_recall_fscore_support



from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM,Dropout , Input , Bidirectional , concatenate
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
#from tensorflow.keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.optimizers import SGD
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
from sklearn.base import BaseEstimator, TransformerMixin
import emoji
import nltk
nltk.download('stopwords')
import json


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def load_training_data_to_pandas(filename = 'train.jsonl'):
    X = []
    Y = []
    fhand = open(filename,encoding='utf8')
    for line in fhand:
        data = json.loads(line)

        lt = data['context']
        lt.reverse()
        fullTweet =   data['response'] + " " + ''.join(lt)

        X.append(fullTweet)
        Y.append(data['label'])
 
    
    return pd.DataFrame({'Tweets': X,'Labels': Y})

def load_test_data_to_pandas(filename = 'test.jsonl'):
    tid = []
    X = []
    Y = []
    fhand = open(filename,encoding='utf8')
    for line in fhand:
        data = json.loads(line)
        tid.append(data['id'])
        lt = data['context']
        lt.reverse()
        fullTweet =   data['response'] + " " + ''.join(lt)

        X.append(fullTweet)
        
    
    return  pd.DataFrame({'ID': tid,
                   'Tweets': X})

In [6]:
twitterdata = load_training_data_to_pandas()
twittertestdata = load_test_data_to_pandas()

In [7]:
#twitterdata = pd.read_csv("dataPandas.csv")
twitterdata.isnull().values.any()
twitterdata.shape

(5000, 2)

In [8]:
class TextCounts(BaseEstimator, TransformerMixin):
    
    def count_regex(self, pattern, tweet):
        return len(re.findall(pattern, tweet))
    
    def fit(self, X, y=None, **fit_params):
        # fit method is used when specific operations need to be done on the train data, but not on the test data
        return self
    
    def transform(self, X, **transform_params):
        count_words = X.apply(lambda x: self.count_regex(r'\w+', x)) 
        count_mentions = X.apply(lambda x: self.count_regex(r'@\w+', x))
        count_hashtags = X.apply(lambda x: self.count_regex(r'#\w+', x))
        count_capital_words = X.apply(lambda x: self.count_regex(r'\b[A-Z]{2,}\b', x))
        count_excl_quest_marks = X.apply(lambda x: self.count_regex(r'!|\?', x))
        count_urls = X.apply(lambda x: self.count_regex(r'http.?://[^\s]+[\s]?', x))
        # We will replace the emoji symbols with a description, which makes using a regex for counting easier
        # Moreover, it will result in having more words in the tweet
        count_emojis = X.apply(lambda x: emoji.demojize(x)).apply(lambda x: self.count_regex(r':[a-z_&]+:', x))
        
        df = pd.DataFrame({'count_words': count_words
                           , 'count_mentions': count_mentions
                           , 'count_hashtags': count_hashtags
                           , 'count_capital_words': count_capital_words
                           , 'count_excl_quest_marks': count_excl_quest_marks
                           , 'count_urls': count_urls
                           , 'count_emojis': count_emojis
                          })
        
        return df
    
tc = TextCounts()
twitterdata_eda = tc.fit_transform(twitterdata.Tweets)
#twitterdata_eda['Labels'] = twitterdata.Labels

In [9]:
twitterdata_meta = twitterdata_eda.to_numpy()
twitterdata_meta.shape

(5000, 7)

In [10]:
class CleanText(BaseEstimator, TransformerMixin):
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)    
   
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

In [11]:
ct = CleanText()
twitterdata_CT = ct.fit_transform(twitterdata.Tweets)
#twitterdata_CT.head()
twitterdata['cTweets'] = twitterdata_CT

In [12]:
X = []
sentences = list(twitterdata['cTweets'])
for sen in sentences:
    #X.append(preprocess_text(sen))
    X.append(sen)

In [13]:
y = twitterdata['Labels']
y = np.array(list(map(lambda x: 1 if x=="SARCASM" else 0, y)))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(twitterdata_eda, y, test_size=0.20, random_state=42)

In [15]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [17]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-12-09 23:27:32--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-09 23:27:32--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-09 23:27:33--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [19]:
import zipfile
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

In [20]:
embeddings_dictionary = dict()
#glove_file = open('./Data/glove.twitter.27B.100d.txt', encoding="utf8")
glove_file = open('glove.6B.100d.txt', encoding="utf8")


for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [21]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [22]:
nlp_input = Input(shape=(maxlen,)) 
meta_input = Input(shape=(7,))
#emb = Embedding(output_dim=embedding_size, input_dim=100, input_length=seq_length)
emb = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)(nlp_input) 
#lstm = LSTM(300, dropout=0.3, recurrent_dropout=0.3)(embed)

nlp_out = Bidirectional(LSTM(128 , dropout=0.3, recurrent_dropout=0.3))(emb) 
concat = concatenate([nlp_out, meta_input]) 
drop = Dropout(0.5)(concat)
#dens = Dense(1)(drop)

#classifier = Dense(32, activation='relu')(drop) 
output = Dense(1, activation='sigmoid')(drop) 
model_blstm = Model(inputs=[nlp_input , meta_input], outputs=[output])

model_blstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_blstm.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     1968800     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256)          234496      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7)]          0                                            
_______________________________________________________________________________________

In [23]:
history = model_blstm.fit([X_train,X_train_meta ], y_train, batch_size=128, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
25/25 [==============================] - 24s 942ms/step - loss: 6.1495 - acc: 0.4959 - val_loss: 3.0134 - val_acc: 0.4600
Epoch 2/50
25/25 [==============================] - 23s 923ms/step - loss: 4.6903 - acc: 0.4875 - val_loss: 2.1598 - val_acc: 0.4675
Epoch 3/50
25/25 [==============================] - 23s 935ms/step - loss: 2.7009 - acc: 0.5238 - val_loss: 1.0577 - val_acc: 0.5750
Epoch 4/50
25/25 [==============================] - 23s 933ms/step - loss: 1.3950 - acc: 0.5456 - val_loss: 0.6737 - val_acc: 0.6500
Epoch 5/50
25/25 [==============================] - 23s 918ms/step - loss: 0.7425 - acc: 0.6309 - val_loss: 0.5902 - val_acc: 0.6725
Epoch 6/50
25/25 [==============================] - 23s 924ms/step - loss: 0.6396 - acc: 0.6581 - val_loss: 0.6331 - val_acc: 0.6438
Epoch 7/50
25/25 [==============================] - 23s 917ms/step - loss: 0.5839 - acc: 0.6878 - val_loss: 0.5390 - val_acc: 0.7050
Epoch 8/50
25/25 [==============================] - 23s 932ms/step - 

In [24]:
score,accuracy = model_blstm.evaluate([X_test,X_test_meta], y_test, verbose=1)

32/32 [==============================] - 2s 62ms/step - loss: 0.8410 - acc: 0.7060


In [25]:
print("Test Score:", score)
print("Test Accuracy:", accuracy)

Test Score: 0.8410437703132629
Test Accuracy: 0.7059999704360962


In [26]:
#twittertestdata = pd.read_csv("dftestdata.csv")
print(twittertestdata.isnull().values.any())
twittertestdata_CT = ct.fit_transform(twittertestdata.Tweets)
twittertestdata_eda = tc.fit_transform(twittertestdata.Tweets)
twittertestdata_meta = twittertestdata_eda.to_numpy()


twittertestdata['cTweets'] = twittertestdata_CT

False


In [27]:
X_val = []
sentences = list(twittertestdata['cTweets'])
for sen in sentences:
    X_val.append(sen)
    
X_validate = tokenizer.texts_to_sequences(X_val)
#X_valTokens = tokenizer.texts_to_sequences(X_validate)
X_valTokens = pad_sequences(X_validate, padding='post', maxlen=maxlen)

In [28]:
validation = model_blstm.predict([X_valTokens,twittertestdata_meta])

In [29]:
twittertestdata['Predict'] = validation

In [30]:
twittertestdata['PLabel'] = np.where(twittertestdata['Predict'] > 0.5, "SARCASM", "NOT_SARCASM")

In [31]:
twittertestdata.head()

,ID,Tweets,cTweets,Predict,PLabel
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...",year old finish read nietzsch ask ayo papa peo...,0.002249,NOT_SARCASM
1,twitter_2,@USER @USER How many verifiable lies has he to...,mani verifi lie told document truth teller sur...,0.993121,SARCASM
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,mayb doc scrub coach mean get hammer gold stan...,0.688716,SARCASM
3,twitter_4,@USER @USER is just a cover up for the real ha...,cover real hate insid left nutshel url hate pl...,0.548973,SARCASM
4,twitter_5,@USER @USER @USER The irony being that he even...,ironi even ask quit articul consid comment fin...,0.987163,SARCASM


In [32]:
twittertestdata.to_csv('answer_fblstm1.txt', columns = ["ID" , "PLabel"] , header = False , index = False)